导入所需最少依赖。

# Search Agent

In [78]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

LOCAL_MODEL_PATH = r"C:/Users/nashk/Documents/nashknight/search_agent/Qwen3-0.6B"

定义并加载最小模型+分词器函数。

In [79]:
def load():
    tok = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH, local_files_only=True)
    m = AutoModelForCausalLM.from_pretrained(LOCAL_MODEL_PATH, local_files_only=True, device_map="auto" if torch.cuda.is_available() else None)
    return m, tok

model, tokenizer = load()

Some parameters are on the meta device because they were offloaded to the cpu.


定义最小生成函数，返回原始与处理后输出。

In [86]:
def generate(prompt, max_new_tokens=512):
    msgs = [{"role":"user","content":prompt}]
    if hasattr(tokenizer, "apply_chat_template"):
        text = tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    else:
        text = prompt
    inp = tokenizer(text, return_tensors="pt")
    for k,v in inp.items(): inp[k] = v.to(model.device)
    ids = model.generate(**inp, max_new_tokens=max_new_tokens)
    gen_ids = ids[0][inp['input_ids'].shape[-1]:]  # 仅保留新生成部分
    raw_text = tokenizer.decode(gen_ids, skip_special_tokens=False)
    clean_text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return gen_ids.tolist(), raw_text, clean_text

执行一次简单测试并输出原始与处理后结果，确保模型导入和基本功能正常。

In [88]:
tokens, raw_text, clean_text = generate("今天我心情很好")
print("RAW_ALL_TOKENS_TEXT:\n", raw_text)
print("\n\nCLEAN_RESPONSE:\n", clean_text)

RAW_ALL_TOKENS_TEXT:
 <think>
好的，用户说今天心情很好。我需要以友好的态度回应。首先，要确认用户的情绪状态，然后表达理解和关心。可以提到今天过得开心，或者一起分享一些快乐的事情。同时，保持语气自然，避免过于机械。可能还需要询问用户是否需要更多帮助或者建议，这样可以促进进一步的交流。要注意用词简单，避免复杂句子，让用户感到轻松愉快。
</think>

今天过得超级开心呀！心情特别好呢～要不要一起分享一下你最近有什么有趣的事情？😊<|im_end|>


CLEAN_RESPONSE:
 <think>
好的，用户说今天心情很好。我需要以友好的态度回应。首先，要确认用户的情绪状态，然后表达理解和关心。可以提到今天过得开心，或者一起分享一些快乐的事情。同时，保持语气自然，避免过于机械。可能还需要询问用户是否需要更多帮助或者建议，这样可以促进进一步的交流。要注意用词简单，避免复杂句子，让用户感到轻松愉快。
</think>

今天过得超级开心呀！心情特别好呢～要不要一起分享一下你最近有什么有趣的事情？😊


In [89]:
# Search Agent v1.0 core
BASE_PROMPT = (
    "你是一个搜索智能体。规则:"\
    "\n1. 能直接回答时直接回答，不用<search>."\
    "\n2. 对于任何需要最新、易变或实时事实的信息（例如股价、汇率、天气、新闻标题等），必须在第一次内部思考阶段使用<search>查询短语</search>标注要检索的内容。"\
    "\n3. 触发关键词（任一出现即必须搜索）：实时、当前、现在、最新、股价、报价、价格、行情、天气、新闻。出现这些词且内容为具体事实时，第1轮思考必须出现<search>标签。"\
    "\n4. <search>内部只放纯查询短语，不放句子，不嵌套其它<search>，例如：<search>特斯拉实时股价</search>；不要写：<search>我要查询特斯拉实时股价</search>。"\
    "\n5. 在未获得搜索结果前，不要在最终回答中给出具体实时数值。"\
    "\n6. 如果已经提供搜索结果，整合后直接输出答案，不要再插入<search>."\
    "\n7. 输出自然中文，不要复述规则。")

INJECTED_RESULT = "特斯拉的当前股价为403.99美元"  # 固定搜索引擎返回占位


def extract_search_query(raw: str):
    # 查找所有 <search> 开始位置与第一个 </search>
    close = re.search(r"</search>", raw)
    if not close:
        return None, None  # 没有闭合标签
    # 所有开标签
    opens = list(re.finditer(r"<search>", raw[:close.start()+8]))
    if not opens:
        return None, None
    start_pos = opens[-1].end()
    end_pos = close.start()
    query = raw[start_pos:end_pos].strip()
    # 如果 query 里再包含 <search> 说明嵌套，取最后一个出现后的内容
    if '<search>' in query:
        query = query.split('<search>')[-1].strip()
    # 过滤过长/含换行的无效内容
    if not query or len(query) > 80:
        return None, None
    raw_trunc = raw[:close.end()]  # 截断到 </search>（包含）
    return query, raw_trunc


def run_search_agent(user_query, max_rounds=5, max_new_tokens=256):
    rounds = []
    prompt = f"{BASE_PROMPT}\n用户: {user_query}"
    last_clean = ""
    search_done = False
    for r in range(1, max_rounds + 1):
        current_query = user_query if r == 1 else (
            f"上一轮回答: {last_clean} | 搜索结果: {INJECTED_RESULT}" if search_done else f"等待搜索: {user_query}")
        _, raw, clean = generate(prompt, max_new_tokens=max_new_tokens)
        entry = {"round": r, "raw": raw, "clean": clean, "query": current_query}
        if not search_done:
            q, raw_trunc = extract_search_query(raw)
            if q:
                entry['raw'] = raw_trunc
                entry['search'] = q
                rounds.append(entry)
                last_clean = clean
                search_done = True
                # 下一轮 prompt：整合搜索结果，禁止再次搜索
                prompt = (
                    f"{BASE_PROMPT}\n用户原问题: {user_query}\n搜索结果: {INJECTED_RESULT}\n请基于搜索结果与原问题直接给出最终答案，不要再使用<search>标签。")
                continue
        # 没有搜索或已搜索完成，认为是最终轮
        rounds.append(entry)
        break
    return rounds


def show_rounds(rounds):
    for i, info in enumerate(rounds, 1):
        print(f"=== Round {info['round']} ===")
        print(f"query: {info.get('query','')}")
        print()
        print('Raw_responses:')
        print(info['raw'])
        if 'search' in info:
            print(f"search_content: {info['search']}")
        print()
        is_last = (i == len(rounds))
        if is_last:
            print('Clean_responses:')
            print(info['clean'])
        else:
            print('Clean_responses: (not final)')
            print('(intermediate thinking; final answer will be in the last round)')
        print()


In [90]:
# Tests for Search Agent v1.0
print("\n### Test 1: 不需要搜索 ###")
rounds1 = run_search_agent("今天我心情真好")
show_rounds(rounds1)



### Test 1: 不需要搜索 ###
=== Round 1 ===
query: 今天我心情真好

Raw_responses:
<think>
好的，用户说今天心情真好，我需要按照规则来处理。首先，规则1是直接回答，但用户没有提供具体信息，所以可能需要进一步询问。不过根据规则，如果用户没有提供具体事实，可能需要先确认是否有需要搜索的内容。但根据规则2，如果需要实时信息，比如股价、天气等，必须在第一次思考时使用<search>标签。用户的话只是表达心情，没有提到具体的数据，所以可能不需要搜索。但需要检查是否有触发关键词的情况。用户没有提到实时、当前、现在等关键词，所以可能不需要搜索。因此，直接回答用户的话，不需要使用<search>标签。
</think>

今天心情真好！<|im_end|>

Clean_responses:
<think>
好的，用户说今天心情真好，我需要按照规则来处理。首先，规则1是直接回答，但用户没有提供具体信息，所以可能需要进一步询问。不过根据规则，如果用户没有提供具体事实，可能需要先确认是否有需要搜索的内容。但根据规则2，如果需要实时信息，比如股价、天气等，必须在第一次思考时使用<search>标签。用户的话只是表达心情，没有提到具体的数据，所以可能不需要搜索。但需要检查是否有触发关键词的情况。用户没有提到实时、当前、现在等关键词，所以可能不需要搜索。因此，直接回答用户的话，不需要使用<search>标签。
</think>

今天心情真好！

=== Round 1 ===
query: 今天我心情真好

Raw_responses:
<think>
好的，用户说今天心情真好，我需要按照规则来处理。首先，规则1是直接回答，但用户没有提供具体信息，所以可能需要进一步询问。不过根据规则，如果用户没有提供具体事实，可能需要先确认是否有需要搜索的内容。但根据规则2，如果需要实时信息，比如股价、天气等，必须在第一次思考时使用<search>标签。用户的话只是表达心情，没有提到具体的数据，所以可能不需要搜索。但需要检查是否有触发关键词的情况。用户没有提到实时、当前、现在等关键词，所以可能不需要搜索。因此，直接回答用户的话，不需要使用<search>标签。
</think>

今天心情真好！<|im_end|>

Clean_respon

In [91]:
print("\n### Test 2: 需要搜索 (特斯拉股价) ###")
rounds2 = run_search_agent("查询特斯拉的实时股价")
show_rounds(rounds2)



### Test 2: 需要搜索 (特斯拉股价) ###
=== Round 1 ===
query: 查询特斯拉的实时股价

Raw_responses:
<think>
好的，用户让我查询特斯拉的实时股价。根据规则，我需要先检查是否有直接回答。但特斯拉的股价通常不是实时的，所以需要使用<search>来查找。规则3提到如果出现关键词实时、当前、现在、最新、股价、报价、价格、行情、天气、新闻，必须用<search>。这里关键词是实时，所以必须搜索。然后，我需要构造一个搜索查询，比如“特斯拉实时股价”。这样就能获取最新的信息了，不需要再提供具体数值，因为规则5说如果已经提供搜索结果就整合输出，不能给出具体数值。所以最终答案应该包含搜索结果，但不包含实际的股价数据。
</think>

<search>特斯拉实时股价</search>
search_content: 特斯拉实时股价

Clean_responses: (not final)
(intermediate thinking; final answer will be in the last round)

=== Round 2 ===
query: 上一轮回答: <think>
好的，用户让我查询特斯拉的实时股价。根据规则，我需要先检查是否有直接回答。但特斯拉的股价通常不是实时的，所以需要使用<search>来查找。规则3提到如果出现关键词实时、当前、现在、最新、股价、报价、价格、行情、天气、新闻，必须用<search>。这里关键词是实时，所以必须搜索。然后，我需要构造一个搜索查询，比如“特斯拉实时股价”。这样就能获取最新的信息了，不需要再提供具体数值，因为规则5说如果已经提供搜索结果就整合输出，不能给出具体数值。所以最终答案应该包含搜索结果，但不包含实际的股价数据。
</think>

<search>特斯拉实时股价</search> | 搜索结果: 特斯拉的当前股价为403.99美元

Raw_responses:
<think>
好的，用户的问题是查询特斯拉的实时股价。首先，我需要检查是否需要使用搜索。根据规则，如果直接回答的话就直接给出，否则需要使用<search>标签。但这里用户已经提供了搜索结果，当前股价是403.99美元。所以不需要再使用<search>标签，直接整合结果即可。确认所有规